In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import seaborn as sns
import re
from IPython.core.display import HTML

In [2]:
traffic = pd.read_csv('../data/crime_nhtsa.csv')
traffic_state = pd.read_csv('../data/traffic_fatalities_state.csv')

In [3]:
traffic['location'] = traffic['zipcode'].apply(lambda x: 'Gulfport Beach' if x == 39501 else ('Isle of Palms' if x == 29451 else 'Panama City Beach' if x == 32407 else 'Dauphin Island' if x == 36528 else 'Orange Beach' if x == 36561 else 'Miramar Beach' if x == 32550 else 'Tybee Island'))

In [4]:
traffic

,zipcode,Crash,2017,2018,2019,2020,2021,"2017 (per 100,000)","2018 (per 100,000)","2019 (per 100,000)","2020 (per 100,000)","2021 (per 100,000)",location
0,32550,Total Fatalities (All Crashes)*,21,24,19,29,39,30.83,33.86,25.63,38.23,48.71,Miramar Beach
1,32550,(1) Alcohol-Impaired Driving (BAC=.08+) Fatali...,10,6,6,9,11,14.68,8.47,8.09,11.86,13.74,Miramar Beach
2,32550,(2) Single Vehicle Crash Fatalities,15,14,10,17,20,22.02,19.75,13.49,22.41,24.98,Miramar Beach
3,32550,(3) Large Truck Involved Crash Fatalities,3,2,1,1,7,4.40,2.82,1.35,1.32,8.74,Miramar Beach
4,32550,(4) Speeding Involved Crash Fatalities,3,5,1,4,1,4.40,7.05,1.35,5.27,1.25,Miramar Beach
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,39501,Passenger Car Occupant Fatalities,12,10,11,12,8,5.85,4.85,5.29,5.75,3.82,Gulfport Beach
87,39501,Light Truck Occupant Fatalities,5,9,9,7,14,2.44,4.36,4.33,3.36,6.69,Gulfport Beach
88,39501,Motorcyclist Fatalities,2,4,4,9,5,0.98,1.94,1.92,4.31,2.39,Gulfport Beach
89,39501,Pedestrian Fatalities,10,13,2,8,10,4.88,6.30,0.96,3.83,4.78,Gulfport Beach


In [5]:
traffic['Crash'] = traffic['Crash'].apply(lambda x: 'Alcohol Impaired' if x == '(1) Alcohol-Impaired Driving (BAC=.08+) Fatalities'\
                                                   else ('Single Vehicle' if x == '(2) Single Vehicle Crash Fatalities'\
                                                        else 'Large Truck Involved' if x == '(3) Large Truck Involved Crash Fatalities'\
                                                        else 'Speeding' if x == '(4) Speeding Involved Crash Fatalities'\
                                                        else 'Rollover' if x == '(5) Rollover Involved Crash Fatalities '\
                                                        else 'Roadway Departure' if x == '(6) Roadway Departure Involved Crash Fatalities'\
                                                        else 'Intersection' if x == '(7) Intersection (or Intersection Related) Crash Fatalities'\
                                                        else 'Bicyclist' if x == 'Bicyclist (or Other Cyclist) Fatalities'\
                                                        else 'Light Truck' if x == 'Light Truck Occupant Fatalities'\
                                                        else 'Motorcyclist' if x == 'Motorcyclist Fatalities'\
                                                        else 'Passenger Car' if x == 'Passenger Car Occupant Fatalities'\
                                                        else 'Pedestrian' if x == 'Pedestrian Fatalities'\
                                                        else 'Total Fatalities'))

In [6]:
traffic = traffic.rename(columns = {'Crash':'crash_type'})

In [7]:
traffic

,zipcode,crash_type,2017,2018,2019,2020,2021,"2017 (per 100,000)","2018 (per 100,000)","2019 (per 100,000)","2020 (per 100,000)","2021 (per 100,000)",location
0,32550,Total Fatalities,21,24,19,29,39,30.83,33.86,25.63,38.23,48.71,Miramar Beach
1,32550,Alcohol Impaired,10,6,6,9,11,14.68,8.47,8.09,11.86,13.74,Miramar Beach
2,32550,Single Vehicle,15,14,10,17,20,22.02,19.75,13.49,22.41,24.98,Miramar Beach
3,32550,Large Truck Involved,3,2,1,1,7,4.40,2.82,1.35,1.32,8.74,Miramar Beach
4,32550,Speeding,3,5,1,4,1,4.40,7.05,1.35,5.27,1.25,Miramar Beach
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,39501,Passenger Car,12,10,11,12,8,5.85,4.85,5.29,5.75,3.82,Gulfport Beach
87,39501,Light Truck,5,9,9,7,14,2.44,4.36,4.33,3.36,6.69,Gulfport Beach
88,39501,Motorcyclist,2,4,4,9,5,0.98,1.94,1.92,4.31,2.39,Gulfport Beach
89,39501,Pedestrian,10,13,2,8,10,4.88,6.30,0.96,3.83,4.78,Gulfport Beach


# Dauphin Traffic

In [8]:
dauphin_traffic = traffic.loc[traffic['location']=='Dauphin Island']

In [9]:
dauphin_traffic = dauphin_traffic.T
dauphin_traffic = dauphin_traffic[1:]
dauphin_traffic = dauphin_traffic.reset_index()
new_header = dauphin_traffic.iloc[0]
dauphin_traffic = dauphin_traffic[1:]
dauphin_traffic = dauphin_traffic[:-1]
dauphin_traffic.columns = new_header

In [10]:
dauphin_traffic.columns = ['crash_type', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'DELETE',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist']
dauphin_traffic = dauphin_traffic.drop(columns = {'DELETE'})

In [11]:
test = dauphin_traffic.iloc[5:]
test['crash_type'] = test['crash_type'].str.extract('(\d{4})')

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\2869244404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crash_type'] = test['crash_type'].str.extract('(\d{4})')


In [12]:
test.columns = ['crash_type', 'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)', 'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)', 'Bicyclist (per 100,000)']

In [13]:
dauphin_traffic = pd.merge(dauphin_traffic, test,
                            on='crash_type', how='inner')
dauphin_traffic = dauphin_traffic.rename(columns={'crash_type':'year'})
dauphin_traffic['location'] = 'Dauphin Island'

# Gulfport

In [14]:
gulfport = traffic.loc[traffic['location']=='Gulfport Beach']

In [15]:
gulfport = gulfport.T
gulfport = gulfport[1:]
gulfport = gulfport.reset_index()
new_header = gulfport.iloc[0]
gulfport = gulfport[1:]
gulfport = gulfport[:-1]
gulfport.columns = new_header

In [16]:
gulfport.columns = ['crash_type', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'DELETE',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist']
gulfport = gulfport.drop(columns = {'DELETE'})

In [17]:
test = gulfport.iloc[5:]
test['crash_type'] = test['crash_type'].str.extract('(\d{4})')

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\364814372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crash_type'] = test['crash_type'].str.extract('(\d{4})')


In [18]:
test.columns = ['crash_type', 'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)', 'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)', 'Bicyclist (per 100,000)']

In [19]:
gulfport = pd.merge(gulfport, test,
                            on='crash_type', how='inner')
gulfport = gulfport.rename(columns={'crash_type':'year'})
gulfport['location'] = 'Gulfport Beach'

# Isle of Palms

In [20]:
palms = traffic.loc[traffic['location']=='Isle of Palms']

In [21]:
palms = palms.T
palms = palms[1:]
palms = palms.reset_index()
new_header = palms.iloc[0]
palms = palms[1:]
palms = palms[:-1]
palms.columns = new_header

In [22]:
palms.columns = ['crash_type', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'DELETE',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist']
palms = palms.drop(columns = {'DELETE'})

In [23]:
test = palms.iloc[5:]
test['crash_type'] = test['crash_type'].str.extract('(\d{4})')

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\2152389809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crash_type'] = test['crash_type'].str.extract('(\d{4})')


In [24]:
test.columns = ['crash_type', 'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)', 'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)', 'Bicyclist (per 100,000)']

In [25]:
palms = pd.merge(palms, test,
                            on='crash_type', how='inner')
palms = palms.rename(columns={'crash_type':'year'})
palms['location'] = 'Isle of Palms'

# Miramar Beach

In [26]:
miramar = traffic.loc[traffic['location']=='Miramar Beach']

In [27]:
miramar = miramar.T
miramar = miramar[1:]
miramar = miramar.reset_index()
new_header = miramar.iloc[0]
miramar = miramar[1:]
miramar = miramar[:-1]
miramar.columns = new_header

In [28]:
miramar.columns = ['crash_type', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'DELETE',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist']
miramar = miramar.drop(columns = {'DELETE'})

In [29]:
test = miramar.iloc[5:]
test['crash_type'] = test['crash_type'].str.extract('(\d{4})')

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\849242878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crash_type'] = test['crash_type'].str.extract('(\d{4})')


In [30]:
test.columns = ['crash_type', 'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)', 'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)', 'Bicyclist (per 100,000)']

In [31]:
miramar = pd.merge(miramar, test,
                            on='crash_type', how='inner')
miramar = miramar.rename(columns={'crash_type':'year'})
miramar['location'] = 'Miramar Beach'

# Orange Beach

In [32]:
orange = traffic.loc[traffic['location']=='Orange Beach']

In [33]:
orange = orange.T
orange = orange[1:]
orange = orange.reset_index()
new_header = orange.iloc[0]
orange = orange[1:]
orange = orange[:-1]
orange.columns = new_header

In [34]:
orange.columns = ['crash_type', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'DELETE',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist']
orange = orange.drop(columns = {'DELETE'})

In [35]:
test = orange.iloc[5:]
test['crash_type'] = test['crash_type'].str.extract('(\d{4})')

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\4049464316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crash_type'] = test['crash_type'].str.extract('(\d{4})')


In [36]:
test.columns = ['crash_type', 'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)', 'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)', 'Bicyclist (per 100,000)']

In [37]:
orange = pd.merge(orange, test,
                            on='crash_type', how='inner')
orange = orange.rename(columns={'crash_type':'year'})
orange['location'] = 'Orange Beach'

# Panama

In [38]:
panama = traffic.loc[traffic['location']=='Panama City Beach']

In [39]:
panama = panama.T
panama = panama[1:]
panama = panama.reset_index()
new_header = panama.iloc[0]
panama = panama[1:]
panama = panama[:-1]
panama.columns = new_header

In [40]:
panama.columns = ['crash_type', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'DELETE',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist']
panama = panama.drop(columns = {'DELETE'})

In [41]:
test = panama.iloc[5:]
test['crash_type'] = test['crash_type'].str.extract('(\d{4})')

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\288995145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crash_type'] = test['crash_type'].str.extract('(\d{4})')


In [42]:
test.columns = ['crash_type', 'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)', 
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)', 'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)', 'Bicyclist (per 100,000)']

In [43]:
panama = pd.merge(panama, test,
                            on='crash_type', how='inner')
panama = panama.rename(columns={'crash_type':'year'})
panama['location'] = 'Panama City Beach'

# Tybee

In [44]:
tybee = traffic.loc[traffic['location']=='Tybee Island']

In [45]:
tybee = tybee.T
tybee = tybee[1:]
tybee = tybee.reset_index()
new_header = tybee.iloc[0]
tybee = tybee[1:]
tybee = tybee[:-1]
tybee.columns = new_header

In [46]:
tybee.columns = ['crash_type', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'DELETE',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist']
tybee = tybee.drop(columns = {'DELETE'})

In [47]:
test = tybee.iloc[5:]
test['crash_type'] = test['crash_type'].str.extract('(\d{4})')

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\3565635771.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['crash_type'] = test['crash_type'].str.extract('(\d{4})')


In [48]:
test.columns = ['crash_type', 'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)', 
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)', 'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)', 'Bicyclist (per 100,000)']

In [49]:
tybee = pd.merge(tybee, test,
                            on='crash_type', how='inner')
tybee = tybee.rename(columns={'crash_type':'year'})
tybee['location'] = 'Tybee Island'

In [50]:
all_traffic = pd.merge(dauphin_traffic, gulfport,
        on = ('year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Total Fatalities',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist',
       'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)',
       'Single Vehicle (per 100,000)', 'Large Truck Involved (per 100,000)',
       'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'), 
         how='outer')
all_traffic = pd.merge(all_traffic, miramar,
        on = ('year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Total Fatalities',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist',
       'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)',
       'Single Vehicle (per 100,000)', 'Large Truck Involved (per 100,000)',
       'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'), 
         how='outer')
all_traffic = pd.merge(all_traffic, orange,
        on = ('year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Total Fatalities',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist',
       'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)',
       'Single Vehicle (per 100,000)', 'Large Truck Involved (per 100,000)',
       'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'), 
         how='outer')
all_traffic = pd.merge(all_traffic, palms,
        on = ('year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Total Fatalities',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist',
       'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)',
       'Single Vehicle (per 100,000)', 'Large Truck Involved (per 100,000)',
       'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'), 
         how='outer')
all_traffic = pd.merge(all_traffic, panama,
        on = ('year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Total Fatalities',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist',
       'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)',
       'Single Vehicle (per 100,000)', 'Large Truck Involved (per 100,000)',
       'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'), 
         how='outer')
all_traffic = pd.merge(all_traffic, tybee,
        on = ('year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Total Fatalities',
       'Roadway Departure', 'Intersection', 'Passenger Car', 'Light Truck',
       'Motorcyclist', 'Pedestrian', 'Bicyclist',
       'Total Fatalities (per 100,000)', 'Alcohol Impaired (per 100,000)',
       'Single Vehicle (per 100,000)', 'Large Truck Involved (per 100,000)',
       'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'), 
         how='outer')

C:\Users\bswil\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\bswil\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\bswil\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\Users\bswil\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when

In [51]:
all_traffic

,year,Total Fatalities,Alcohol Impaired,Single Vehicle,Large Truck Involved,Speeding,Roadway Departure,Intersection,Passenger Car,Light Truck,...,"Large Truck Involved (per 100,000)","Speeding (per 100,000)","Roadway Departure (per 100,000)","Intersection (per 100,000)","Passenger Car (per 100,000)","Light Truck (per 100,000)","Motorcyclist (per 100,000)","Pedestrian (per 100,000)","Bicyclist (per 100,000)",location
0,2017,75.0,19.0,41.0,7.0,14.0,36.0,17.0,27.0,22.0,...,1.69,3.38,8.69,4.1,6.52,5.31,1.93,2.66,0.24,Dauphin Island
1,2018,74.0,21.0,37.0,8.0,17.0,44.0,11.0,30.0,22.0,...,1.93,4.1,10.62,2.66,7.24,5.31,2.17,2.41,0.0,Dauphin Island
2,2019,84.0,28.0,44.0,13.0,18.0,45.0,11.0,25.0,29.0,...,3.14,4.35,10.88,2.66,6.05,7.01,2.66,3.63,0.24,Dauphin Island
3,2020,61.0,16.0,33.0,7.0,13.0,27.0,13.0,16.0,21.0,...,1.69,3.14,6.52,3.14,3.86,5.07,1.45,3.62,0.0,Dauphin Island
4,2021,79.0,29.0,37.0,11.0,24.0,45.0,13.0,31.0,28.0,...,2.66,5.81,10.89,3.15,7.5,6.78,0.48,3.39,0.24,Dauphin Island
5,2017,33.0,9.0,22.0,0.0,3.0,12.0,6.0,12.0,5.0,...,0.0,1.46,5.85,2.93,5.85,2.44,0.98,4.88,0.0,Gulfport Beach
6,2018,38.0,8.0,25.0,2.0,2.0,11.0,8.0,10.0,9.0,...,0.97,0.97,5.33,3.88,4.85,4.36,1.94,6.3,0.48,Gulfport Beach
7,2019,28.0,8.0,17.0,1.0,9.0,16.0,3.0,11.0,9.0,...,0.48,4.33,7.7,1.44,5.29,4.33,1.92,0.96,0.96,Gulfport Beach
8,2020,38.0,7.0,23.0,4.0,5.0,13.0,11.0,12.0,7.0,...,1.92,2.4,6.23,5.27,5.75,3.36,4.31,3.83,0.0,Gulfport Beach
9,2021,39.0,9.0,21.0,3.0,5.0,11.0,6.0,8.0,14.0,...,1.43,2.39,5.25,2.87,3.82,6.69,2.39,4.78,0.48,Gulfport Beach


# Aggregation

In [52]:
all_traffic.columns

Index(['year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Roadway Departure', 'Intersection',
       'Passenger Car', 'Light Truck', 'Motorcyclist', 'Pedestrian',
       'Bicyclist', 'Total Fatalities (per 100,000)',
       'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'],
      dtype='object')

In [53]:
traffic_agg = all_traffic.groupby(['location'])['Total Fatalities (per 100,000)',
       'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)'].mean()
traffic_agg = traffic_agg.reset_index()

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\4195934309.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  traffic_agg = all_traffic.groupby(['location'])['Total Fatalities (per 100,000)',


In [54]:
alcohol = traffic_agg['Alcohol Impaired (per 100,000)'].rank(method='min').tolist()
single = traffic_agg['Single Vehicle (per 100,000)'].rank(method='min').tolist()
large = traffic_agg['Large Truck Involved (per 100,000)'].rank(method='min').tolist()
speeding = traffic_agg['Speeding (per 100,000)'].rank(method='min').tolist()
roadway = traffic_agg['Roadway Departure (per 100,000)'].rank(method='min').tolist()
intersection = traffic_agg['Intersection (per 100,000)'].rank(method='min').tolist()
passenger = traffic_agg['Passenger Car (per 100,000)'].rank(method='min').tolist()
light = traffic_agg['Light Truck (per 100,000)'].rank(method='min').tolist()
motor = traffic_agg['Motorcyclist (per 100,000)'].rank(method='min').tolist()
pedestrian = traffic_agg['Pedestrian (per 100,000)'].rank(method='min').tolist()
bike = traffic_agg['Bicyclist (per 100,000)'].rank(method='min').tolist()
loc = traffic_agg['location'].tolist()

In [55]:
traffic_locations_ranked = pd.DataFrame({'location': loc, 'alcohol_impaired_rank': alcohol, 'single_vehicle_rank': single, 'large_truck_involved_rank': large,
                                 'speeding_rank': speeding, 'roadway_departure_rank': roadway, 'intersection_rank': intersection,
                                 'passenger_car_rank': passenger, 'light_truck_rank': light, 'motorcyclist_rank':motor,
                                    'pedestrian_rank':pedestrian, 'bicyclist_rank':bike})

In [56]:
traffic_locations_ranked['average_rank'] = (traffic_locations_ranked['alcohol_impaired_rank']+traffic_locations_ranked['single_vehicle_rank']+traffic_locations_ranked['large_truck_involved_rank']+traffic_locations_ranked['speeding_rank']+traffic_locations_ranked['roadway_departure_rank']+traffic_locations_ranked['intersection_rank']+traffic_locations_ranked['passenger_car_rank']+traffic_locations_ranked['light_truck_rank']+traffic_locations_ranked['motorcyclist_rank']+traffic_locations_ranked['pedestrian_rank']+traffic_locations_ranked['bicyclist_rank'])/11

In [57]:
travel_time = ['7:00', '7:30', '8:45', '7:00', '7:10', '7:45', '8:10']

In [58]:
traffic_locations_ranked['travel_time'] = travel_time
traffic_locations_ranked['travel_time'] = pd.to_datetime(traffic_locations_ranked['travel_time'])
traffic_locations_ranked['travel_time'] = traffic_locations_ranked['travel_time'].dt.time

In [59]:
traffic_locations_ranked

,location,alcohol_impaired_rank,single_vehicle_rank,large_truck_involved_rank,speeding_rank,roadway_departure_rank,intersection_rank,passenger_car_rank,light_truck_rank,motorcyclist_rank,pedestrian_rank,bicyclist_rank,average_rank,travel_time
0,Dauphin Island,5.0,3.0,6.0,6.0,6.0,2.0,6.0,6.0,3.0,3.0,1.0,4.272727,07:00:00
1,Gulfport Beach,2.0,6.0,1.0,2.0,3.0,4.0,4.0,3.0,4.0,5.0,4.0,3.454545,07:30:00
2,Isle of Palms,6.0,5.0,2.0,7.0,4.0,5.0,5.0,2.0,5.0,6.0,5.0,4.727273,08:45:00
3,Miramar Beach,7.0,7.0,7.0,5.0,7.0,7.0,7.0,7.0,7.0,7.0,6.0,6.727273,07:00:00
4,Orange Beach,3.0,2.0,4.0,4.0,5.0,1.0,3.0,5.0,1.0,1.0,3.0,2.909091,07:10:00
5,Panama City Beach,4.0,4.0,5.0,1.0,2.0,6.0,2.0,4.0,6.0,4.0,7.0,4.090909,07:45:00
6,Tybee Island,1.0,1.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,2.0,2.0,1.818182,08:10:00


In [60]:
traffic_agg

,location,"Total Fatalities (per 100,000)","Alcohol Impaired (per 100,000)","Single Vehicle (per 100,000)","Large Truck Involved (per 100,000)","Speeding (per 100,000)","Roadway Departure (per 100,000)","Intersection (per 100,000)","Passenger Car (per 100,000)","Light Truck (per 100,000)","Motorcyclist (per 100,000)","Pedestrian (per 100,000)","Bicyclist (per 100,000)"
0,Dauphin Island,18.026,5.462,9.280,2.222,4.156,9.520,3.142,6.234,5.896,1.738,3.142,0.144
1,Gulfport Beach,16.964,3.956,10.418,0.960,2.310,6.072,3.278,5.112,4.236,2.308,4.150,0.384
2,Isle of Palms,17.798,5.866,10.166,1.274,6.892,6.948,3.770,5.238,3.960,2.492,4.744,0.876
3,Miramar Beach,35.452,11.368,20.530,3.726,3.864,19.096,6.722,11.542,9.918,5.094,4.892,1.066
4,Orange Beach,14.782,4.520,7.572,1.692,3.796,8.454,1.942,4.808,5.554,1.700,1.620,0.362
5,Panama City Beach,18.864,4.704,9.814,2.018,0.992,5.796,6.248,4.664,4.610,3.238,3.990,1.470
6,Tybee Island,11.698,2.938,7.048,1.566,3.144,4.104,3.216,3.632,2.738,1.706,3.008,0.274


# Cleaning for Tableau

In [61]:
test_1 = traffic_agg.loc[traffic_agg['location']=='Dauphin Island'][['Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)']].T.reset_index()
test_1.columns = ['type', 'per_100k']
test_1['location'] = 'Dauphin Island'

test_2 = traffic_agg.loc[traffic_agg['location']=='Tybee Island'][['Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)']].T.reset_index()
test_2.columns = ['type', 'per_100k']
test_2['location'] = 'Tybee Island'

test_3 = traffic_agg.loc[traffic_agg['location']=='Orange Beach'][['Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)']].T.reset_index()
test_3.columns = ['type', 'per_100k']
test_3['location'] = 'Orange Beach'

test_7 = traffic_agg.loc[traffic_agg['location']=='Miramar Beach'][['Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)']].T.reset_index()
test_7.columns = ['type', 'per_100k']
test_7['location'] = 'Miramar Beach'

test_4 = traffic_agg.loc[traffic_agg['location']=='Isle of Palms'][['Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)']].T.reset_index()
test_4.columns = ['type', 'per_100k']
test_4['location'] = 'Isle of Palms'

test_5 = traffic_agg.loc[traffic_agg['location']=='Gulfport Beach'][['Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)']].T.reset_index()
test_5.columns = ['type', 'per_100k']
test_5['location'] = 'Gulfport Beach'

test_6 = traffic_agg.loc[traffic_agg['location']=='Panama City Beach'][['Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)']].T.reset_index()
test_6.columns = ['type', 'per_100k']
test_6['location'] = 'Panama City Beach'

In [62]:
test = pd.merge(test_1, test_2,
        on = ('type', 'per_100k', 'location'), how='outer')
test = pd.merge(test, test_3,
        on = ('type', 'per_100k', 'location'), how='outer')
test = pd.merge(test, test_4,
        on = ('type', 'per_100k', 'location'), how='outer')
test = pd.merge(test, test_5,
        on = ('type', 'per_100k', 'location'), how='outer')
test = pd.merge(test, test_6,
        on = ('type', 'per_100k', 'location'), how='outer')
test = pd.merge(test, test_7,
        on = ('type', 'per_100k', 'location'), how='outer')
test

,type,per_100k,location
0,"Alcohol Impaired (per 100,000)",5.462,Dauphin Island
1,"Single Vehicle (per 100,000)",9.280,Dauphin Island
2,"Large Truck Involved (per 100,000)",2.222,Dauphin Island
3,"Speeding (per 100,000)",4.156,Dauphin Island
4,"Roadway Departure (per 100,000)",9.520,Dauphin Island
...,...,...,...
72,"Passenger Car (per 100,000)",11.542,Miramar Beach
73,"Light Truck (per 100,000)",9.918,Miramar Beach
74,"Motorcyclist (per 100,000)",5.094,Miramar Beach
75,"Pedestrian (per 100,000)",4.892,Miramar Beach


In [63]:
test[['type', 'delete']] = test['type'].str.split('\(', expand=True)
test = test.drop(columns = 'delete')

In [64]:
test

,type,per_100k,location
0,Alcohol Impaired,5.462,Dauphin Island
1,Single Vehicle,9.280,Dauphin Island
2,Large Truck Involved,2.222,Dauphin Island
3,Speeding,4.156,Dauphin Island
4,Roadway Departure,9.520,Dauphin Island
...,...,...,...
72,Passenger Car,11.542,Miramar Beach
73,Light Truck,9.918,Miramar Beach
74,Motorcyclist,5.094,Miramar Beach
75,Pedestrian,4.892,Miramar Beach


In [65]:
rank_1 = traffic_locations_ranked.loc[traffic_locations_ranked['location']=='Dauphin Island'][['alcohol_impaired_rank', 'single_vehicle_rank',
       'large_truck_involved_rank', 'speeding_rank', 'roadway_departure_rank',
       'intersection_rank', 'passenger_car_rank', 'light_truck_rank',
       'motorcyclist_rank', 'pedestrian_rank', 'bicyclist_rank']].T.reset_index()
rank_1.columns = ['type', 'rank']
rank_1['location'] = 'Dauphin Island'

rank_2 = traffic_locations_ranked.loc[traffic_locations_ranked['location']=='Tybee Island'][['alcohol_impaired_rank', 'single_vehicle_rank',
       'large_truck_involved_rank', 'speeding_rank', 'roadway_departure_rank',
       'intersection_rank', 'passenger_car_rank', 'light_truck_rank',
       'motorcyclist_rank', 'pedestrian_rank', 'bicyclist_rank']].T.reset_index()
rank_2.columns = ['type', 'rank']
rank_2['location'] = 'Tybee Island'

rank_3 = traffic_locations_ranked.loc[traffic_locations_ranked['location']=='Orange Beach'][['alcohol_impaired_rank', 'single_vehicle_rank',
       'large_truck_involved_rank', 'speeding_rank', 'roadway_departure_rank',
       'intersection_rank', 'passenger_car_rank', 'light_truck_rank',
       'motorcyclist_rank', 'pedestrian_rank', 'bicyclist_rank']].T.reset_index()
rank_3.columns = ['type', 'rank']
rank_3['location'] = 'Orange Beach'

rank_4 = traffic_locations_ranked.loc[traffic_locations_ranked['location']=='Miramar Beach'][['alcohol_impaired_rank', 'single_vehicle_rank',
       'large_truck_involved_rank', 'speeding_rank', 'roadway_departure_rank',
       'intersection_rank', 'passenger_car_rank', 'light_truck_rank',
       'motorcyclist_rank', 'pedestrian_rank', 'bicyclist_rank']].T.reset_index()
rank_4.columns = ['type', 'rank']
rank_4['location'] = 'Miramar Beach'

rank_5 = traffic_locations_ranked.loc[traffic_locations_ranked['location']=='Gulfport Beach'][['alcohol_impaired_rank', 'single_vehicle_rank',
       'large_truck_involved_rank', 'speeding_rank', 'roadway_departure_rank',
       'intersection_rank', 'passenger_car_rank', 'light_truck_rank',
       'motorcyclist_rank', 'pedestrian_rank', 'bicyclist_rank']].T.reset_index()
rank_5.columns = ['type', 'rank']
rank_5['location'] = 'Gulfport Beach'

rank_6 = traffic_locations_ranked.loc[traffic_locations_ranked['location']=='Panama City Beach'][['alcohol_impaired_rank', 'single_vehicle_rank',
       'large_truck_involved_rank', 'speeding_rank', 'roadway_departure_rank',
       'intersection_rank', 'passenger_car_rank', 'light_truck_rank',
       'motorcyclist_rank', 'pedestrian_rank', 'bicyclist_rank']].T.reset_index()
rank_6.columns = ['type', 'rank']
rank_6['location'] = 'Panama City Beach'

rank_7 = traffic_locations_ranked.loc[traffic_locations_ranked['location']=='Isle of Palms'][['alcohol_impaired_rank', 'single_vehicle_rank',
       'large_truck_involved_rank', 'speeding_rank', 'roadway_departure_rank',
       'intersection_rank', 'passenger_car_rank', 'light_truck_rank',
       'motorcyclist_rank', 'pedestrian_rank', 'bicyclist_rank']].T.reset_index()
rank_7.columns = ['type', 'rank']
rank_7['location'] = 'Isle of Palms'

In [66]:
rank = pd.merge(rank_1, rank_2,
               on=('type', 'rank', 'location'), how='outer')
rank = pd.merge(rank, rank_3,
               on=('type', 'rank', 'location'), how='outer')
rank = pd.merge(rank, rank_4,
               on=('type', 'rank', 'location'), how='outer')
rank = pd.merge(rank, rank_5,
               on=('type', 'rank', 'location'), how='outer')
rank = pd.merge(rank, rank_6,
               on=('type', 'rank', 'location'), how='outer')
rank = pd.merge(rank, rank_7,
               on=('type', 'rank', 'location'), how='outer')

In [67]:
rank[['type', 'delete', 'again', 'yep']] = rank['type'].str.split('\_', expand=True)
rank = rank.drop(columns = 'delete')
rank = rank.drop(columns = 'again')
rank = rank.drop(columns = 'yep')

In [68]:
rank['type'] = rank['type'].str.title()
rank

,type,rank,location
0,Alcohol,5.0,Dauphin Island
1,Single,3.0,Dauphin Island
2,Large,6.0,Dauphin Island
3,Speeding,6.0,Dauphin Island
4,Roadway,6.0,Dauphin Island
...,...,...,...
72,Passenger,5.0,Isle of Palms
73,Light,2.0,Isle of Palms
74,Motorcyclist,5.0,Isle of Palms
75,Pedestrian,6.0,Isle of Palms


In [69]:
for index, row in rank.iterrows():
    if row.type == 'Alcohol':
        rank.loc[index, 'type'] = 'Alcohol Impaired'
    elif row.type == 'Single':
        rank.loc[index, 'type'] = 'Single Vehicle'
    elif row.type == 'Large':
        rank.loc[index, 'type'] = 'Large Truck Involved'
    elif row.type == 'Roadway':
        rank.loc[index, 'type'] = 'Roadway Departure'
    elif row.type == 'Passenger':
        rank.loc[index, 'type'] = 'Passenger Car'
    elif row.type == 'Light':
        rank.loc[index, 'type'] = 'Light Truck'
    

In [70]:
traffic_cleaned = pd.merge(rank, test,
                          on = ('type', 'location'), how = 'outer')

In [71]:
traffic_cleaned

,type,rank,location,per_100k
0,Alcohol Impaired,5.0,Dauphin Island,NaN
1,Single Vehicle,3.0,Dauphin Island,NaN
2,Large Truck Involved,6.0,Dauphin Island,NaN
3,Speeding,6.0,Dauphin Island,NaN
4,Roadway Departure,6.0,Dauphin Island,NaN
...,...,...,...,...
149,Passenger Car,NaN,Miramar Beach,11.542
150,Light Truck,NaN,Miramar Beach,9.918
151,Motorcyclist,NaN,Miramar Beach,5.094
152,Pedestrian,NaN,Miramar Beach,4.892


In [72]:
traffic_state['2010'] = traffic_state['2010'].str.replace(',', '').astype(int)
traffic_state['2011'] = traffic_state['2011'].str.replace(',', '').astype(int)
traffic_state['2012'] = traffic_state['2012'].str.replace(',', '').astype(int)
traffic_state['2013'] = traffic_state['2013'].str.replace(',', '').astype(int)
traffic_state['2014'] = traffic_state['2014'].str.replace(',', '').astype(int)
traffic_state['2015'] = traffic_state['2015'].str.replace(',', '').astype(int)
traffic_state['2016'] = traffic_state['2016'].str.replace(',', '').astype(int)
traffic_state['2017'] = traffic_state['2017'].str.replace(',', '').astype(int)
traffic_state['2018'] = traffic_state['2018'].str.replace(',', '').astype(int)
traffic_state['2019'] = traffic_state['2019'].str.replace(',', '').astype(int)
traffic_state['2020'] = traffic_state['2020'].str.replace(',', '').astype(int)

In [73]:
traffic_state

,State,type,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Alabama,Highway Fatalities,862,895,865,853,820,850,1083,948,953,930,934,NaN,NaN
1,Alabama,Rail fatalities,17,17,10,18,21,15,12,24,15,15,13,21.0,20.0
2,Alabama,Recreational boating fatalities,20,19,17,10,13,21,14,21,17,28,19,10.0,NaN
3,Alabama,Transit Fatalities,0,0,0,0,1,0,0,0,0,0,0,0.0,NaN
4,Florida,Highway Fatalities,2444,2400,2431,2403,2494,2938,3176,3116,3135,3185,3331,NaN,NaN
5,Florida,Rail fatalities,44,32,30,35,34,44,44,64,47,55,39,60.0,71.0
6,Florida,Recreational boating fatalities,69,61,50,58,70,52,70,66,57,62,72,61.0,NaN
7,Florida,Transit Fatalities,14,12,13,15,8,8,6,8,7,10,7,17.0,NaN
8,Georgia,Highway Fatalities,1247,1226,1192,1180,1164,1432,1556,1540,1505,1492,1664,NaN,NaN
9,Georgia,Rail fatalities,29,19,21,28,25,24,23,19,29,23,35,21.0,19.0


# AVG Fatalities per Year

In [74]:
all_traffic['Total Fatalities'].sum()/5

310.2

In [75]:
crime = pd.read_csv('../data/crime_locations_rank.csv')
crime_data = pd.read_csv('../data/beach_crime.csv')

In [76]:
crime = crime.drop(columns = {'Unnamed: 0'})

In [77]:
crime

,location,murder_rank,rapes_rank,robberies_rank,assaults_rank,burglaries_rank,thefts_rank,auto_thefts_rank,arson_rank,average_rank
0,Dauphin Island,1.0,5.0,2.0,5.0,5.0,3.0,4.0,1.0,3.250
1,Gulfport Beach,7.0,4.0,6.0,4.0,6.0,6.0,7.0,7.0,5.875
2,Isle of Palms,3.0,1.0,1.0,1.0,4.0,2.0,5.0,3.0,2.500
3,Miramar Beach,5.0,3.0,5.0,6.0,3.0,1.0,6.0,5.0,4.250
4,Orange Beach,4.0,6.0,3.0,3.0,2.0,5.0,3.0,2.0,3.500
5,Panama City Beach,6.0,7.0,7.0,7.0,7.0,7.0,1.0,6.0,6.000
6,Tybee Island,1.0,2.0,4.0,2.0,1.0,4.0,2.0,4.0,2.500


In [78]:
traffic_locations_ranked

,location,alcohol_impaired_rank,single_vehicle_rank,large_truck_involved_rank,speeding_rank,roadway_departure_rank,intersection_rank,passenger_car_rank,light_truck_rank,motorcyclist_rank,pedestrian_rank,bicyclist_rank,average_rank,travel_time
0,Dauphin Island,5.0,3.0,6.0,6.0,6.0,2.0,6.0,6.0,3.0,3.0,1.0,4.272727,07:00:00
1,Gulfport Beach,2.0,6.0,1.0,2.0,3.0,4.0,4.0,3.0,4.0,5.0,4.0,3.454545,07:30:00
2,Isle of Palms,6.0,5.0,2.0,7.0,4.0,5.0,5.0,2.0,5.0,6.0,5.0,4.727273,08:45:00
3,Miramar Beach,7.0,7.0,7.0,5.0,7.0,7.0,7.0,7.0,7.0,7.0,6.0,6.727273,07:00:00
4,Orange Beach,3.0,2.0,4.0,4.0,5.0,1.0,3.0,5.0,1.0,1.0,3.0,2.909091,07:10:00
5,Panama City Beach,4.0,4.0,5.0,1.0,2.0,6.0,2.0,4.0,6.0,4.0,7.0,4.090909,07:45:00
6,Tybee Island,1.0,1.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,2.0,2.0,1.818182,08:10:00


In [79]:
safety_rating = pd.merge(traffic_locations_ranked, crime,
        on='location', how='inner')

In [80]:
safety_rating = safety_rating[['location', 'average_rank_x', 'average_rank_y']]
safety_rating.columns=['location', 'avg_rank_traffic', 'avg_rank_crime']

In [81]:
safety_rating['safety_rating'] = (safety_rating['avg_rank_traffic']*.2)+(safety_rating['avg_rank_crime']*.8)

In [82]:
safety_rating

,location,avg_rank_traffic,avg_rank_crime,safety_rating
0,Dauphin Island,4.272727,3.250,3.454545
1,Gulfport Beach,3.454545,5.875,5.390909
2,Isle of Palms,4.727273,2.500,2.945455
3,Miramar Beach,6.727273,4.250,4.745455
4,Orange Beach,2.909091,3.500,3.381818
5,Panama City Beach,4.090909,6.000,5.618182
6,Tybee Island,1.818182,2.500,2.363636


In [83]:
crime_data = crime_data.drop(columns = {'Unnamed: 0'})

In [84]:
crime_data.groupby(['location'])['Murders', 'Murders (per 100,000)', 'Rapes',
       'Rapes (per 100,000)', 'Robberies', 'Robberies (per 100,000)',
       'Assaults', 'Assaults (per 100,000)', 'Burglaries',
       'Burglaries (per 100,000)', 'Thefts', 'Thefts (per 100,000)',
       'Auto Thefts', 'Auto Thefts (per 100,000)', 'Arson',
       'Arson (per 100,000)'].mean()

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\2913119154.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  crime_data.groupby(['location'])['Murders', 'Murders (per 100,000)', 'Rapes',


,Murders,"Murders (per 100,000)",Rapes,"Rapes (per 100,000)",Robberies,"Robberies (per 100,000)",Assaults,"Assaults (per 100,000)",Burglaries,"Burglaries (per 100,000)",Thefts,"Thefts (per 100,000)",Auto Thefts,"Auto Thefts (per 100,000)",Arson,"Arson (per 100,000)"
location,,,,,,,,,,,,,,,,
Dauphin Island,0.000000,0.000000,0.428571,33.885714,0.142857,10.214286,2.428571,187.214286,13.571429,1021.664286,37.857143,2907.835714,2.071429,155.992857,0.000000,0.000000
Gulfport Beach,6.857143,9.742857,23.000000,32.757143,85.071429,122.450000,107.142857,151.057143,747.285714,1077.757143,2764.571429,3946.928571,200.428571,286.985714,18.714286,26.678571
Isle of Palms,0.071429,1.635714,1.071429,24.328571,0.357143,8.028571,2.928571,66.621429,30.357143,688.085714,101.000000,2291.000000,8.142857,184.242857,0.071429,1.671429
Miramar Beach,4.857143,3.842857,36.357143,28.242857,139.285714,111.135714,280.571429,220.150000,682.785714,554.335714,1912.142857,1502.685714,279.428571,220.378571,8.785714,7.092857
Orange Beach,0.153846,2.638462,3.230769,53.092308,1.000000,16.853846,8.538462,143.569231,32.307692,543.638462,206.769231,3488.538462,6.692308,112.700000,0.076923,1.376923
Panama City Beach,0.857143,6.735714,15.642857,120.035714,21.714286,168.957143,68.857143,530.392857,159.428571,1213.728571,884.214286,6816.857143,5.214286,40.728571,1.785714,13.428571
Tybee Island,0.000000,0.000000,0.857143,25.828571,0.571429,17.300000,3.642857,110.300000,8.071429,252.457143,94.714286,2968.142857,2.928571,90.828571,0.071429,1.892857


In [85]:
crime_data.columns

Index(['Year', 'Crime Index', 'Murders', 'Murders (per 100,000)', 'Rapes',
       'Rapes (per 100,000)', 'Robberies', 'Robberies (per 100,000)',
       'Assaults', 'Assaults (per 100,000)', 'Burglaries',
       'Burglaries (per 100,000)', 'Thefts', 'Thefts (per 100,000)',
       'Auto Thefts', 'Auto Thefts (per 100,000)', 'Arson',
       'Arson (per 100,000)', 'location'],
      dtype='object')

In [86]:
crime['average_rank'] = (crime['murder_rank']*.15)+(crime['rapes_rank']*.15)+(crime['robberies_rank']*.10)+(crime['assaults_rank']*.15)+(crime['burglaries_rank']*.15)+(crime['thefts_rank']*.20)+(crime['auto_thefts_rank']*.07)+(crime['arson_rank']*.03)

In [87]:
crime_total_rank = crime[['location', 'average_rank']]

In [88]:
crime_total_rank = crime_total_rank.rename(columns = {'average_rank':'total_crime_rank'})

In [89]:
all_traffic.groupby(['location'])['Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Roadway Departure', 'Intersection',
       'Passenger Car', 'Light Truck', 'Motorcyclist', 'Pedestrian',
       'Bicyclist'].mean()

C:\Users\bswil\AppData\Local\Temp\ipykernel_24676\3123355960.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  all_traffic.groupby(['location'])['Alcohol Impaired', 'Single Vehicle',


,Alcohol Impaired,Single Vehicle,Large Truck Involved,Speeding,Roadway Departure,Intersection,Passenger Car,Light Truck,Motorcyclist,Pedestrian,Bicyclist
location,,,,,,,,,,,
Dauphin Island,22.6,38.4,9.2,17.2,39.4,13.0,25.8,24.4,7.2,13.0,0.6
Gulfport Beach,8.2,21.6,2.0,4.8,12.6,6.8,10.6,8.8,4.8,8.6,0.8
Isle of Palms,24.0,41.6,5.2,28.2,28.4,15.4,21.4,16.2,10.2,19.4,3.6
Miramar Beach,8.4,15.2,2.8,2.8,14.2,5.0,8.6,7.4,3.8,3.6,0.8
Orange Beach,10.2,17.0,3.8,8.6,19.0,4.4,10.8,12.6,3.8,3.6,0.8
Panama City Beach,8.4,17.6,3.6,1.8,10.4,11.2,8.4,8.2,5.8,7.2,2.6
Tybee Island,8.6,20.6,4.6,9.2,12.0,9.4,10.6,8.0,5.0,8.8,0.8


In [90]:
all_traffic.columns

Index(['year', 'Total Fatalities', 'Alcohol Impaired', 'Single Vehicle',
       'Large Truck Involved', 'Speeding', 'Roadway Departure', 'Intersection',
       'Passenger Car', 'Light Truck', 'Motorcyclist', 'Pedestrian',
       'Bicyclist', 'Total Fatalities (per 100,000)',
       'Alcohol Impaired (per 100,000)', 'Single Vehicle (per 100,000)',
       'Large Truck Involved (per 100,000)', 'Speeding (per 100,000)',
       'Roadway Departure (per 100,000)', 'Intersection (per 100,000)',
       'Passenger Car (per 100,000)', 'Light Truck (per 100,000)',
       'Motorcyclist (per 100,000)', 'Pedestrian (per 100,000)',
       'Bicyclist (per 100,000)', 'location'],
      dtype='object')

In [91]:
traffic_locations_ranked['average_rank'] = (traffic_locations_ranked['alcohol_impaired_rank']*.12)+(traffic_locations_ranked['single_vehicle_rank']*.12)+(traffic_locations_ranked['large_truck_involved_rank']*.03)+(traffic_locations_ranked['speeding_rank']*.12)+(traffic_locations_ranked['roadway_departure_rank']*.12)+(traffic_locations_ranked['intersection_rank']*.10)+(traffic_locations_ranked['passenger_car_rank']*.12)+(traffic_locations_ranked['light_truck_rank']*.05)+(traffic_locations_ranked['motorcyclist_rank']*.05)+(traffic_locations_ranked['pedestrian_rank']*.12)+(traffic_locations_ranked['bicyclist_rank']*.05)

In [92]:
traffic_total_rank = traffic_locations_ranked[['location', 'average_rank']]

In [93]:
traffic_total_rank = traffic_total_rank.rename(columns={'average_rank':'total_traffic_rank'})

In [94]:
new_rankings = pd.merge(traffic_total_rank, crime_total_rank,
                       on = 'location', how='outer')

In [95]:
new_rankings['safety_rating'] = (new_rankings['total_traffic_rank']*.2)+(new_rankings['total_crime_rank']*.8)
new_rankings

,location,total_traffic_rank,total_crime_rank,safety_rating
0,Dauphin Island,4.36,3.51,3.680
1,Gulfport Beach,3.62,5.65,5.244
2,Isle of Palms,5.12,2.29,2.856
3,Miramar Beach,6.71,3.82,4.398
4,Orange Beach,2.83,3.82,3.622
5,Panama City Beach,3.64,6.40,5.848
6,Tybee Island,1.72,2.36,2.232
